# This script is meant for the processing of different files into a format that's ready for filtering by expression
# and downstream analyses

In [12]:
# Import packages
import scanpy as sc
import anndata as ad
import pandas as pd
import leidenalg
import time

ModuleNotFoundError: No module named 'leidenalg'

# Function definitions start here

In [2]:
#Time wrapper
def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} executed in {end_time - start_time} seconds")
        return result
    return wrapper

In [3]:
# DEFAULT QC VALUES. Calibrated to Sarah Teichmann's paper "Cells of the human intestinal tract mapped across space and time." These QC values will apply by default for this entire script.
def filter_cells_for_UMAP(data, min_ct = 2000, min_gen = 500, min_cell = 3, mt_pct = 60, max_genes = 0, normed = 0, d_score = 0.24): 
    adata = data # This is to avoid writing into the file that's entered as an argument
    print("################# Filtering ... #################")
    sc.pp.filter_cells(adata, min_counts = min_ct) # Filter cells based on number of RNA reads
    sc.pp.filter_cells(adata, min_genes= min_gen) # Filter cells based on the number of recognized genes
    sc.pp.filter_genes(adata, min_cells = min_cell) # Filter genes based on the minimum number of cells expressing it
    adata_prefilt = adata[adata.obs['doublet_scores'] < 0.24]
    if max_genes > 0:
        adata_prefilt = adata_prefilt[adata_prefilt.obs['n_genes_by_counts'] < max_genes]
        
    if not normed:
        adata_filt = adata_prefilt[adata_prefilt.obs['pct_counts_mt'] < mt_pct] # Filtering based on percentage of mitochondrial genes
    else:
        adata_filt = adata_prefilt
    return adata_filt    

In [4]:
@time_it
def process_for_UMAP(data, leiden_res = 0.8, filtering = 1, min_ct = 2000, min_gen = 500, min_cell = 3, mt_pct = 60, max_genes = 0, normed = 0, d_score = 0.24): # DEFAULT QC VALUES
    adata = data # This is to avoid writing into the file that's entered as an argument
    if filtering:
        adata_filt = filter_cells_for_UMAP(data = adata, min_ct = min_ct, min_gen = min_gen, min_cell = min_cell, max_genes = max_genes, mt_pct = mt_pct, d_score = d_score)
    else:
        adata_filt = adata       
    print("################# Normalizing ... #################")
    sc.pp.normalize_total(adata_filt, target_sum=1e4) # Normalize
    print("################# Log scaling ... #################")
    sc.pp.log1p(adata_filt) # Log scaling
    print("################# Finding variable genes ... #################")
    sc.pp.highly_variable_genes(adata_filt, min_mean = 0.0125, max_mean = 3, min_disp = 0.5) # Compute differentially expressed genes within the sample
    print("################# Saving raw data ... #################")
    adata_filt.raw = adata_filt # Store the raw files in its own layer
    print("################# Filtering on variable genes ... #################")
    adata_filt = adata_filt[:, adata_filt.var.highly_variable] # Filter on genes that are highly variable
    print("################# Regressing ... #################")
    sc.pp.regress_out(adata_filt, ['total_counts', 'pct_counts_mt']) # Regression. Not sure what that is.
    print("################# Scaling ... #################")
    sc.pp.scale(adata_filt, max_value = 10) # Scale the data
    print("################# Calculating PCA ... #################")
    sc.tl.pca(adata_filt, svd_solver='arpack') # Compute PCA
    print("################# Calculating tSNE ... #################")
    sc.tl.tsne(adata_filt) # Calculate tsne
    print("################# Calculating neighbors ... #################")
    sc.pp.neighbors(adata_filt) # Calculate neighbors
    print("################# Calculating Leiden ... #################")
    sc.tl.leiden(adata_filt, resolution = leiden_res) # Calculate Leiden clusters
    print("################# Calculating PAGA ... #################")
    sc.tl.paga(adata_filt) # Calculate PAGA
    print("################# Plotting PAGA ... #################")
    sc.pl.paga(adata_filt, plot = 1)  # remove `plot=False` if you want to see the coarse-grained graph
    print("################# Calculating UMAP init_pos = paga #################")
    sc.tl.umap(adata_filt, init_pos='paga') # Plot PAGA
    print("################# Calculating UMAP ... #################")
    sc.tl.umap(adata_filt) # Calculate UMAP
    print("#################Plotting UMAP ... #################")
    sc.pl.umap(adata_filt, color = ['leiden']) # Plot UMAP and show Leiden clusters
    return adata_filt
#######################################################
################## FUNCTION DEF END ###################
#######################################################

In [5]:
@time_it
def recalc_UMAP(data_filt, leiden_res = 0.8):
    adata_filt = data_filt
    sc.tl.pca(adata_filt, svd_solver='arpack') # Compute PCA
    print("################# Calculating tSNE ... #################")
    sc.tl.tsne(adata_filt) # Calculate tsne
    print("################# Calculating neighbors ... #################")
    sc.pp.neighbors(adata_filt) # Calculate neighbors
    print("################# Calculating Leiden ... #################")
    sc.tl.leiden(adata_filt, resolution = leiden_res) # Calculate Leiden clusters
    print("################# Calculating PAGA ... #################")
    sc.tl.paga(adata_filt) # Calculate PAGA
    print("################# Plotting PAGA ... #################")
    sc.pl.paga(adata_filt, plot = 1)  # remove `plot=False` if you want to see the coarse-grained graph
    print("################# Calculating UMAP init_pos = paga#################")
    sc.tl.umap(adata_filt, init_pos='paga') # Calculate PAGA
    print("################# Calculating UMAP ... #################")
    sc.tl.umap(adata_filt) # Calculate UMAP
    print("################# Plotting UMAP ... #################")
    sc.pl.umap(adata_filt, color = ['leiden']) # Plot UMAP and show Leiden clusters
    return adata_filt
#######################################################
################## FUNCTION DEF END ###################
#######################################################

In [6]:
def isolate_cells_by_gene(data, gene, threshold):
    # Now subset_ant_mt_filt contains only the highly variable genes
    data_subset = data[data[:, gene].X > threshold]
    return data_subset
#######################################################
################## FUNCTION DEF END ###################
#######################################################

In [7]:
# This function filters the leiden clusters that are positivefor the gene you specify
# It assumes that you already did the differential expression analysis. 
# diff is boolean specifying if differential expresion is already done
# threshold is the threshold of expression
def filter_clusters_by_gene(data, gene, threshold = 0.5):
    # Load your AnnData object
    adata = data
    sc.tl.rank_genes_groups(adata, groupby='leiden')
    # Extract the DataFrame for the differential expression results
    de_results = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    # Define a threshold for significant expression (adjust as needed)
    expression_threshold = threshold
    # Find clusters with significant gene expression
    significant_clusters = []
    for cluster in de_results.columns:
        gene_presence = de_results[cluster].str.contains(gene)
        gene_expression = adata.uns['rank_genes_groups']['logfoldchanges'][cluster][gene_presence]
        if any(gene_expression >= expression_threshold):
            significant_clusters.append(cluster)
    # Subset the data to include only cells from the significant clusters
    adata_subset = adata[adata.obs['leiden'].isin(significant_clusters)].copy()
    return adata_subset

# Setting enviroment and other variables

In [8]:
#%% Environment settings and misc variables
sc.settings.verbosity = 3
sc.set_figure_params(dpi = 600)

# MIK67 = Ki67, TNSFRSF19 = TROY
inspect_stem = ['LGR5', 'MKI67', 'TNFRSF19', 'BMI1', 'LRIG1', 'leiden', 'Localization']
global_res = 0.5
start_time = time.time()

Read the files

In [9]:
col_org_unfilt = sc.read("C:/Work cache/Project sync/PhD/Research projects/AGR2 follow-up/Data cache/ssRNAseq/Aline/raw_data/agr2colon_organoids_unfilt.h5ad")
ant_unfilt = sc.read("C:/Work cache/Project sync/PhD/Research projects/AGR2 follow-up/Data cache/ssRNAseq/Aline/raw_data/agr2_unfilt_antrum.h5ad")
duo_unfilt = sc.read("C:/Work cache/Project sync/PhD/Research projects/AGR2 follow-up/Data cache/ssRNAseq/Aline/raw_data/agr2_unfilt_duodenum.h5ad")
col_unfilt = sc.read("C:/Work cache/Project sync/PhD/Research projects/AGR2 follow-up/Data cache/ssRNAseq/Aline/raw_data/agr2_unfilt_colon.h5ad")

C:\Users\alsha\anaconda3\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
C:\Users\alsha\anaconda3\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
C:\Users\alsha\anaconda3\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
C:\Users\alsha\anaconda3\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


Joining and creating new lables

In [10]:
combined = ad.concat([ant_unfilt, duo_unfilt, col_unfilt], join = 'outer')
combined_unique = combined[~combined.obs['cellbarcode'].duplicated(keep = 'first')].copy()
combined_nocol = ad.concat([ant_unfilt, duo_unfilt], join = 'outer')
combined.obs['Localization'] = combined.obs['Site'].astype(str) + ' ' + combined.obs['Patient'].astype(str)
combined_nocol = ad.concat([ant_unfilt, duo_unfilt], join = 'outer')
combined_nocol.obs['Localization'] = combined_nocol.obs['Site'].astype(str) + ' ' + combined_nocol.obs['Patient'].astype(str)
ant_unfilt.obs['Localization'] = ant_unfilt.obs['Site'].astype(str) + ' ' + ant_unfilt.obs['Patient'].astype(str)
combined_control = combined_unique[combined_unique.obs['Patient'] == 'GI6253', :] # Using the unique object seems to work for filtering while usign the combined object directly doesn't. No idea why.
combined_control.obs['Localization'] = combined_control.obs['Site'].astype(str) + ' ' + combined_control.obs['Patient'].astype(str)
combined_patient = combined_unique[combined_unique.obs['Patient'] == 'P26', :] # Using the unique object seems to work for filtering while usign the combined object directly doesn't. No idea why.
combined_patient.obs['Localization'] = combined_patient.obs['Site'].astype(str) + ' ' + combined_patient.obs['Patient'].astype(str)
duo_unfilt.obs['Localization'] = duo_unfilt.obs['Site'].astype(str) + ' ' + duo_unfilt.obs['Patient'].astype(str)
duo_pat_unfilt = duo_unfilt[duo_unfilt.obs['Patient'] == 'P26', :]
duo_pat_unique = duo_pat_unfilt[~duo_pat_unfilt.obs['cellbarcode'].duplicated(keep = 'first')].copy()
duo_cont_unfilt = duo_unfilt[duo_unfilt.obs['Patient'] == 'GI6253', :]
duo_cont_unique = duo_cont_unfilt[~duo_cont_unfilt.obs['cellbarcode'].duplicated(keep = 'first')].copy()

C:\Users\alsha\AppData\Local\Temp\ipykernel_22832\1015900654.py:9: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  combined_control.obs['Localization'] = combined_control.obs['Site'].astype(str) + ' ' + combined_control.obs['Patient'].astype(str)
C:\Users\alsha\AppData\Local\Temp\ipykernel_22832\1015900654.py:11: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  combined_patient.obs['Localization'] = combined_patient.obs['Site'].astype(str) + ' ' + combined_patient.obs['Patient'].astype(str)


Initial UMAP processing

In [11]:
antrum_proc = process_for_UMAP(ant_unfilt, leiden_res = global_res)

################# Filtering ... #################
filtered out 2874 cells that have less than 2000 counts
filtered out 464 cells that have less than 500 genes expressed
filtered out 14511 genes that are detected in less than 3 cells
################# Normalizing ... #################
normalizing counts per cell
    finished (0:00:00)
################# Log scaling ... #################


C:\Users\alsha\anaconda3\Lib\site-packages\scanpy\preprocessing\_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


################# Finding variable genes ... #################
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
################# Saving raw data ... #################
################# Filtering on variable genes ... #################
################# Regressing ... #################
regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use


C:\Users\alsha\anaconda3\Lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
C:\Users\alsha\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


    finished (0:00:08)
################# Scaling ... #################
################# Calculating PCA ... #################
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:00)
################# Calculating tSNE ... #################
computing tSNE
    using 'X_pca' with n_pcs = 50
    using sklearn.manifold.TSNE
    finished: added
    'X_tsne', tSNE coordinates (adata.obsm) (0:00:15)
################# Calculating neighbors ... #################
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:07)
################# Calculating Leiden ... #################


ImportError: Please install the leiden algorithm: `conda install -c conda-forge leidenalg` or `pip3 install leidenalg`.

sc.pl.umap(combined_proc, color = 'leiden')

combined_proc = process_for_UMAP(combined, leiden_res = global_res)
combined_nocol_proc = process_for_UMAP(combined_nocol, leiden_res = global_res)
combined_control_proc = process_for_UMAP(combined_control, leiden_res = global_res)
combined_patient_proc = process_for_UMAP(combined_patient, leiden_res = global_res)
duo_proc = process_for_UMAP(duo_unfilt, leiden_res = global_res)
duo_pat_proc = process_for_UMAP(duo_pat_unique, leiden_res = global_res)
duo_cont_proc = process_for_UMAP(duo_cont_unique, leiden_res = global_res)